In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
# @title For environments other than Google Colab
%pip install unsloth vllm

In [ ]:
# @title For Loading Checkpoints from Google Drive (on colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title For setting up dataset in Kaggle
!git clone https://github.com/shuhanmirza/Bengali-Poem-Dataset.git
%cd /kaggle/working/Bengali-Poem-Dataset

In [ ]:
# @title For setting up dataset in Kaggle
import os
import pandas as pd

dataset_dir = 'dataset'

data = []

for poet in os.listdir(dataset_dir):
    poet_dir = os.path.join(dataset_dir, poet)
    if os.path.isdir(poet_dir):
        for poem in os.listdir(poet_dir):
            poem_dir = os.path.join(poet_dir, poem)
            if os.path.isdir(poem_dir):
                class_text = None
                class_file_path = os.path.join(poem_dir, 'CLASS.txt')
                if os.path.exists(class_file_path):
                    with open(class_file_path, 'r', encoding='utf-8') as class_file:
                        class_text = class_file.read().strip()

                for file in os.listdir(poem_dir):
                    if file.endswith('.txt') and file not in ['CLASS.txt', 'SOURCE.txt']:
                        file_path = os.path.join(poem_dir, file)
                        with open(file_path, 'r', encoding='utf-8') as f:
                            poem_text = f.read()
                            title = os.path.splitext(file)[0]
                            data.append({
                                'poet': poet,
                                'category': class_text,
                                'title': title,
                                'poem': poem_text
                            })

df = pd.DataFrame(data)

df.to_csv('poems_dataset.csv', index=False, encoding='utf-8')
!mv /kaggle/working/Bengali-Poem-Dataset/poems_dataset.csv /kaggle/working/
%cd /kaggle/working/
df.head()


In [ ]:
# @title Loading the last checkpoint from Google drive
!cp /content/drive/MyDrive/adapter_model.safetensors /content/outputs/checkpoint-1500/

In [ ]:
# @title Loading the Gemma3 4 billion Parameters using unsloth
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-06 13:26:42 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.3. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
# @title To apply Parameter-Efficient Fine-Tuning (PEFT) to the pre-trained Gemma-3-4b model (unsloth provided)
model = FastModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

Unsloth: Making `base_model.model.vision_tower.vision_model` require gradients


In [ ]:
# @title Prompt and Dataset Loading (upload the poems_dataset.csv) to Google colab
prompt = """You are tasked with writing a poem related to the title in the style mentioned below. The poem should fit the specified category.

### Title:
{}
### Category:
{}

### Poem:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    titles    = examples["title"]
    categorys = examples["category"]
    poems     = examples["poem"]
    texts = []
    for title, category, poem in zip(titles, categorys, poems):
        text = prompt.format(title, category, poem) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/poems_dataset.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6080 [00:00<?, ? examples/s]

In [ ]:
# @title Set up a training pipeline using the UnslothTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

max_seq_length = 2048

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        report_to = "wandb",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/6080 [00:00<?, ? examples/s]

In [ ]:
# @title Start training. Use trainer.train() to start from Zero
trainer.train(resume_from_checkpoint="/content/outputs/checkpoint-1500")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,080 | Num Epochs = 3 | Total steps = 2,280
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 148,099,072/4,000,000,000 (3.70% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ankiporel (ankiporel-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1501,4.881900
1502,4.726600
1503,4.978800
1504,4.942400
1505,4.267300
1506,4.720700
1507,4.375300
1508,5.176300
1509,4.611100
1510,5.043900


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
# @title wandb login setup (needs your API key)
!wandb login
import wandb
wandb.init()

In [ ]:
# @title Save the model
model.save_pretrained("gemma-poetry-bn")
tokenizer.save_pretrained("gemma-poetry-bn")

In [ ]:
# @title Huggingface login
from huggingface_hub import login
login()

In [ ]:
# @title Publish to Huggingface
! huggingface-cli upload Ankita-Porel/gemma3-4b-v1 gemma-poetry-bn

In [ ]:
# @title Load the model from huggingface for inference
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name="Ankita-Porel/gemma3-4b-v1",
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit = False,
    full_finetuning = False,
    # token="hf_..."
)

In [ ]:
# @title Run the tests
model = FastModel.for_inference(model)

prompt = """You are tasked with writing a poem related to the title in the style mentioned below. The poem should fit the specified category.

### Title:
{}
### Category:
{}

### Poem:
{}"""

inputs = tokenizer(
[
    prompt.format(
        "বাংলার বায়ু, বাংলার ফল- পূণ্য হউক, পূণ্য হউক,",
        "কীর্তন",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, repetition_penalty = 2.0, use_cache = True)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

with open("generated_output.txt", "w", encoding="utf-8") as file:
    file.write(generated_text)

print("Output saved to 'output.txt'")
tokenizer.batch_decode(outputs)